In [41]:
maze = [
    
[1,0,0,0,0,0,0,0,0,0,0,"B"],
[1,0,1,1,1,1,1,1,1,1,1,0],
[1,0,1,0,0,0,0,0,0,0,1,0],
[1,0,1,0,1,1,1,1,1,0,1,0],
[1,0,0,0,1,0,0,0,0,0,1,0],
[1,1,1,0,1,0,1,1,1,1,1,0],
["A",0,0,0,1,0,0,0,0,0,0,0],
]


In [42]:
import heapq

# --- Style and Color Definitions ---
class Colors:
    """A class to hold ANSI color codes for styling terminal output."""
    RESET = '\033[0m'
    BOLD = '\033[1m'
    DIM = '\033[2m'
    
    # Text Colors
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'



# --- The Complete MazeSolver Class ---
class MazeSolver:
    def __init__(self, maze):
        self.maze = maze
        self.start, self.goal = self.find_positions(maze)

        print("\n--- Manhattan Heuristic Visualization (Distance to 'B') ---")
        self.manhattanMaze = self.visualize_manhattan()
        self.print_matrix_styled(self.manhattanMaze)
        
        # Solve the maze using the heuristic
        self.path, self.cost = self.a_star() # MODIFIED: Capture cost

        
        if self.path:
            print("--- Solved Path ---")
            self.updated_manhattan(self.path)
            self.print_matrix_styled(self.manhattanMaze)

            # --- NEW: Print Analysis ---
            print(f"--- Analysis ---")
            print(f"{Colors.BOLD}🔹 Path Cost:{Colors.RESET} {self.cost} steps")
            print(f"{Colors.BOLD}🔹 Is Path Optimal?:{Colors.RESET} {Colors.GREEN}Yes{Colors.RESET}")
            print(f"{Colors.DIM}The A* algorithm with an admissible heuristic (like Manhattan distance on a grid) guarantees the shortest path.{Colors.RESET}\n")


    def manhattan(self, a, b):
        """Calculates the Manhattan distance between two points."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def get_neighbors(self, pos):
        """Gets the valid, walkable neighbors of a given position."""
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
        rows, cols = len(self.maze), len(self.maze[0])
        for dx, dy in directions:
            nx, ny = pos[0] + dx, pos[1] + dy
            if 0 <= nx < rows and 0 <= ny < cols and self.maze[nx][ny] != 1:
                neighbors.append((nx, ny))
        return neighbors
    
    def find_positions(self, maze):
        """Finds the start ('A') and goal ('B') positions in the maze."""
        start = goal = None
        for i, row in enumerate(maze):
            for j, cell in enumerate(row):
                if cell == 'A':
                    start = (i, j)
                elif cell == 'B':
                    goal = (i, j)
        return start, goal

    def a_star(self):
        """
        Performs the A* search algorithm to find the shortest path.
        Returns the path and its cost.
        """
        if not self.start or not self.goal:
            print("Start or goal not found in the maze.")
            return [], None

        visited = set()
        came_from = {}
        # The heap stores tuples of: (f_score, node, g_score)
        # f_score = g_score + h_score (total estimated cost)
        # g_score = cost from start to current node
        heap = []
        # Push the start node with a g_score of 0
        heapq.heappush(heap, (self.manhattan(self.start, self.goal), self.start, 0))

        while heap:
            _, current, g_score = heapq.heappop(heap)

            if current == self.goal:
                # Goal reached, reconstruct the path
                path = []
                curr = self.goal
                while curr != self.start:
                    path.append(curr)
                    curr = came_from.get(curr)
                path.append(self.start)
                path.reverse()
                return path, g_score  # MODIFIED: Return the final path and its cost

            if current in visited:
                continue
            visited.add(current)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    # Calculate scores for the neighbor
                    new_g_score = g_score + 1
                    h_score = self.manhattan(neighbor, self.goal)
                    f_score = new_g_score + h_score
                    # self.array.append( f_score) # For analysis if needed
                    
                    heapq.heappush(heap, (f_score, neighbor, new_g_score))
                    came_from[neighbor] = current
        
        # If the loop finishes without finding the goal
        print("No path found.")
        return [], None # MODIFIED: Return None for cost if no path exists
    
    def visualize_manhattan(self):
        """Creates a matrix showing the Manhattan distance from each cell to the goal."""
        rows, cols = len(self.maze), len(self.maze[0])
        dist_matrix = []
        for i in range(rows):
            row = []
            for j in range(cols):
                if self.maze[i][j] == 1:
                    row.append("█")
                elif (i, j) == self.start:
                    row.append('A')
                elif (i, j) == self.goal:
                    row.append('B')
                else:
                    row.append(self.manhattan((i, j), self.goal))
            dist_matrix.append(row)
        return dist_matrix

    def updated_manhattan(self, path):
        """Updates the visualization matrix to show the final path."""
        for (x, y) in path:
            if self.manhattanMaze[x][y] not in ('A', 'B'):
                self.manhattanMaze[x][y] = '*'
    
    def print_matrix_styled(self, matrix):
        """Prints the maze matrix with a game-like border and colors."""
        rows, cols = len(matrix), len(matrix[0])

        print(f"{Colors.BOLD}{Colors.WHITE}╔{'═' * (cols * 3 + 1)}╗{Colors.RESET}")

        for row in matrix:
            print(f"{Colors.BOLD}{Colors.WHITE}║ {Colors.RESET}", end="")
            for val in row:
                cell_str = f"{str(val):<2}"
                
                if val == 'A':
                    print(f"{Colors.BOLD}{Colors.GREEN}{cell_str}{Colors.RESET} ", end="")
                elif val == 'B':
                    print(f"{Colors.BOLD}{Colors.YELLOW}{cell_str}{Colors.RESET} ", end="")
                elif val == '█':
                    print(f"{Colors.BLUE}{'█ '}{Colors.RESET} ", end="")
                elif val == '*':
                    print(f"{Colors.BOLD}{Colors.CYAN}{'• '}{Colors.RESET} ", end="")
                else: # Heuristic numbers
                    print(f"{Colors.DIM}{cell_str}{Colors.RESET} ", end="")
            
            print(f"{Colors.BOLD}{Colors.WHITE}║{Colors.RESET}")

        print(f"{Colors.BOLD}{Colors.WHITE}╚{'═' * (cols * 3 + 1)}╝{Colors.RESET}")
        print()


solver = MazeSolver(maze)


--- Manhattan Heuristic Visualization (Distance to 'B') ---
╔═════════════════════════════════════╗
║ █  10 9  8  7  6  5  4  3  2  1  B  ║
║ █  11 █  █  █  █  █  █  █  █  █  1  ║
║ █  12 █  10 9  8  7  6  5  4  █  2  ║
║ █  13 █  11 █  █  █  █  █  5  █  3  ║
║ █  14 13 12 █  10 9  8  7  6  █  4  ║
║ █  █  █  13 █  11 █  █  █  █  █  5  ║
║ A  16 15 14 █  12 11 10 9  8  7  6  ║
╚═════════════════════════════════════╝

--- Solved Path ---
╔═════════════════════════════════════╗
║ █  •  •  •  •  •  •  •  •  •  •  B  ║
║ █  •  █  █  █  █  █  █  █  █  █  1  ║
║ █  •  █  10 9  8  7  6  5  4  █  2  ║
║ █  •  █  11 █  █  █  █  █  5  █  3  ║
║ █  •  •  •  █  10 9  8  7  6  █  4  ║
║ █  █  █  •  █  11 █  █  █  █  █  5  ║
║ A  •  •  •  █  12 11 10 9  8  7  6  ║
╚═════════════════════════════════════╝

--- Analysis ---
🔹 Path Cost: 21 steps
🔹 Is Path Optimal?: Yes
The A* algorithm with an admissible heuristic (like Manhattan distance on a grid) guarantees the shortest path.



In [43]:
class MazeSolverInflated(MazeSolver):
    """
    Inherits from MazeSolver but uses an "inflated" heuristic for A*.
    This makes the algorithm greedier, often finding a path faster but
    sacrificing the guarantee of finding the shortest path.
    """
    def __init__(self, maze):
        # We override __init__ to provide a custom analysis for this specific case.
        self.maze = maze
        self.start, self.goal = self.find_positions(maze)

        print("\n--- Manhattan Heuristic Visualization (Standard) ---")
        self.manhattanMaze = self.visualize_manhattan()
        self.print_matrix_styled(self.manhattanMaze)
        
        # This call will use the overridden a_star method from THIS class.
        self.path, self.cost = self.a_star()
        
        if self.path:
            print("--- Solved Path (using Inflated Heuristic) ---")
            self.updated_manhattan(self.path)
            self.print_matrix_styled(self.manhattanMaze)

            # --- MODIFIED: Analysis for the Inflated Heuristic ---
            print(f"--- Analysis ---")
            print(f"{Colors.BOLD}🔹 Path Cost:{Colors.RESET} {self.cost} steps")
            print(f"{Colors.BOLD}🔹 Is Heuristic Admissible?:{Colors.RESET} {Colors.YELLOW}No{Colors.RESET}")
            print(f"{Colors.DIM}The heuristic is inflated (h' = 1.5 * h). An admissible heuristic must never overestimate the true cost, but this one can.{Colors.RESET}")
            print(f"{Colors.BOLD}🔹 Is Path Optimal?:{Colors.RESET} {Colors.YELLOW}Not Guaranteed{Colors.RESET}")
            print(f"{Colors.DIM}Because the heuristic is not admissible, A* loses its guarantee of finding the shortest path. It acts more like a Greedy Best-First Search.{Colors.RESET}\n")

    def a_star(self): # Override the a_star method
        """
        A* search using an inflated heuristic (h_score * 1.5).
        Returns the path and its cost.
        """
        if not self.start or not self.goal:
            print("Start or goal not found in the maze.")
            return [], None

        visited = set()
        came_from = {}
        heap = []
        
        # --- HEURISTIC MODIFICATION ---
        h_score = 1.5 * self.manhattan(self.start, self.goal)
        # The heap stores: (f_score, node, g_score)
        heapq.heappush(heap, (h_score, self.start, 0))
        
        final_cost = None

        while heap:
            _, current, g_score = heapq.heappop(heap)

            if current == self.goal:
                final_cost = g_score
                break # Goal found

            if current in visited:
                continue
            visited.add(current)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    new_g_score = g_score + 1
                    # --- HEURISTIC MODIFICATION ---
                    h_score = 1.5 * self.manhattan(neighbor, self.goal)
                    f_score = new_g_score + h_score
                    heapq.heappush(heap, (f_score, neighbor, new_g_score))

                    came_from[neighbor] = current
        
        # If the loop finished without finding the goal
        if final_cost is None:
            print("No path found.")
            return [], None

        # Reconstruct path
        path = []
        curr = self.goal
        while curr != self.start:
            path.append(curr)
            curr = came_from.get(curr)
            if curr is None: # Should not happen if final_cost is set
                return [], None
        path.append(self.start)
        path.reverse()
        
        # Return the found path and its actual cost
        return path, final_cost

print(f"{Colors.BOLD}--- RUNNING CASE 1: INFLATED HEURISTIC ---{Colors.RESET}")
solver_inflated = MazeSolverInflated(maze)



--- RUNNING CASE 1: INFLATED HEURISTIC ---

--- Manhattan Heuristic Visualization (Standard) ---
╔═════════════════════════════════════╗
║ █  10 9  8  7  6  5  4  3  2  1  B  ║
║ █  11 █  █  █  █  █  █  █  █  █  1  ║
║ █  12 █  10 9  8  7  6  5  4  █  2  ║
║ █  13 █  11 █  █  █  █  █  5  █  3  ║
║ █  14 13 12 █  10 9  8  7  6  █  4  ║
║ █  █  █  13 █  11 █  █  █  █  █  5  ║
║ A  16 15 14 █  12 11 10 9  8  7  6  ║
╚═════════════════════════════════════╝

--- Solved Path (using Inflated Heuristic) ---
╔═════════════════════════════════════╗
║ █  •  •  •  •  •  •  •  •  •  •  B  ║
║ █  •  █  █  █  █  █  █  █  █  █  1  ║
║ █  •  █  10 9  8  7  6  5  4  █  2  ║
║ █  •  █  11 █  █  █  █  █  5  █  3  ║
║ █  •  •  •  █  10 9  8  7  6  █  4  ║
║ █  █  █  •  █  11 █  █  █  █  █  5  ║
║ A  •  •  •  █  12 11 10 9  8  7  6  ║
╚═════════════════════════════════════╝

--- Analysis ---
🔹 Path Cost: 21 steps
🔹 Is Heuristic Admissible?: No
The heuristic is inflated (h' = 1.5 * h). An admissible heuristi

In [44]:
# --- CASE 2: INCONSISTENT HEURISTIC (MODIFIED CLASS) ---
class MazeSolverInconsistent(MazeSolver):
    """
    Inherits from MazeSolver but uses an inconsistent (but admissible) heuristic.
    A heuristic is inconsistent if the triangle inequality is violated, meaning the
    heuristic cost can drop by more than the actual cost of moving between nodes.
    """
    def __init__(self, maze):
        # We override __init__ to provide a custom analysis for this case.
        self.maze = maze
        self.start, self.goal = self.find_positions(maze)

        print("\n--- Heuristic Visualization (Standard Manhattan) ---")
        self.manhattanMaze = self.visualize_manhattan()
        self.print_matrix_styled(self.manhattanMaze)
        
        # This call will use the overridden a_star method from THIS class.
        self.path, self.cost = self.a_star()
        
        if self.path:
            print("--- Solved Path (using Inconsistent Heuristic) ---")
            self.updated_manhattan(self.path)
            self.print_matrix_styled(self.manhattanMaze)

            # --- MODIFIED: Analysis for the Inconsistent Heuristic ---
            print(f"--- Analysis ---")
            print(f"{Colors.BOLD}🔹 Path Cost:{Colors.RESET} {self.cost} steps")
            print(f"{Colors.BOLD}🔹 Is Heuristic Admissible?:{Colors.RESET} {Colors.GREEN}Yes{Colors.RESET}")
            print(f"{Colors.DIM}The heuristic never overestimates the true cost to the goal, so it is admissible.{Colors.RESET}")
            print(f"{Colors.BOLD}🔹 Is Heuristic Consistent?:{Colors.RESET} {Colors.YELLOW}No{Colors.RESET}")
            print(f"{Colors.DIM}The heuristic is inconsistent. For example, moving from (2,1) to (2,2), the heuristic value drops dramatically, violating the rule that h(n) <= cost(n, n') + h(n').{Colors.RESET}")
            print(f"{Colors.BOLD}🔹 Is Path Optimal?:{Colors.RESET} {Colors.YELLOW}Not Guaranteed{Colors.RESET}")
            print(f"{Colors.DIM}An A* implementation that prevents revisiting nodes (like this one) is not guaranteed to find the optimal path with an inconsistent heuristic.{Colors.RESET}\n")

    def inconsistent_heuristic(self, pos):
        """
        An admissible but inconsistent heuristic.
        Goal is at (4,5). The Manhattan distance from (2,2) to (4,5) is 5.
        We create a "sinkhole" by giving (4,2) an artificially low heuristic value.
        """
        if pos == (4, 2):
            return 1000 # Deceptively optimistic value
        return self.manhattan(pos, self.goal)
    
    def a_star(self): # Override the a_star method
        """
        A* search using the inconsistent heuristic.
        Returns the path and its cost.
        """
        if not self.start or not self.goal:
            print("Start or goal not found in the maze.")
            return [], None

        visited = set()
        came_from = {}
        heap = []
        
        # --- HEURISTIC MODIFICATION ---
        h_score = self.inconsistent_heuristic(self.start)
        # The heap stores: (f_score, node, g_score)
        heapq.heappush(heap, (h_score, self.start, 0))
        
        final_cost = None

        while heap:
            _, current, g_score = heapq.heappop(heap)

            if current == self.goal:
                final_cost = g_score
                break # Goal found

            if current in visited:
                continue
            visited.add(current)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    new_g_score = g_score + 1
                    # --- HEURISTIC MODIFICATION ---
                    h_score = self.inconsistent_heuristic(neighbor)
                    f_score = new_g_score + h_score
                    heapq.heappush(heap, (f_score, neighbor, new_g_score))
                    came_from[neighbor] = current
        
        # If the loop finished without finding the goal
        if final_cost is None:
            print("No path found.")
            return [], None

        # Reconstruct path
        path = []
        curr = self.goal
        while curr != self.start:
            path.append(curr)
            curr = came_from.get(curr)
            if curr is None:
                return [], None
        path.append(self.start)
        path.reverse()
        
        # Return the found path and its actual cost
        return path, final_cost
print(f"{Colors.BOLD}--- RUNNING CASE 2: INCONSISTENT HEURISTIC ---{Colors.RESET}")
solver_inconsistent = MazeSolverInconsistent(maze)

--- RUNNING CASE 2: INCONSISTENT HEURISTIC ---

--- Heuristic Visualization (Standard Manhattan) ---
╔═════════════════════════════════════╗
║ █  10 9  8  7  6  5  4  3  2  1  B  ║
║ █  11 █  █  █  █  █  █  █  █  █  1  ║
║ █  12 █  10 9  8  7  6  5  4  █  2  ║
║ █  13 █  11 █  █  █  █  █  5  █  3  ║
║ █  14 13 12 █  10 9  8  7  6  █  4  ║
║ █  █  █  13 █  11 █  █  █  █  █  5  ║
║ A  16 15 14 █  12 11 10 9  8  7  6  ║
╚═════════════════════════════════════╝

--- Solved Path (using Inconsistent Heuristic) ---
╔═════════════════════════════════════╗
║ █  10 9  8  7  6  5  4  3  2  1  B  ║
║ █  11 █  █  █  █  █  █  █  █  █  •  ║
║ █  12 █  •  •  •  •  •  •  •  █  •  ║
║ █  13 █  •  █  █  █  █  █  •  █  •  ║
║ █  14 13 •  █  •  •  •  •  •  █  •  ║
║ █  █  █  •  █  •  █  █  █  █  █  •  ║
║ A  •  •  •  █  •  •  •  •  •  •  •  ║
╚═════════════════════════════════════╝

--- Analysis ---
🔹 Path Cost: 33 steps
🔹 Is Heuristic Admissible?: Yes
The heuristic never overestimates the true cost to the 